# Stedin Distribution Transformer - Loss Model
In this notebook we extend the non-linear circuit-coupled transformer model developed in a different notebook. The goal here is to build a loss model of the transformer (core and winding loss). This loss model is evaluated using two tests: 
1. open-circuit test for core losses, and
2. short-circuit test for winding losses.

In [1]:
using gmsh

using LinearAlgebra, SparseArrays
using WriteVTK

using BenchmarkTools

# Geometry Definition
See exercise on Gmsh.

In [2]:
gmsh.finalize()
gmsh.initialize()

In [3]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the y-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Mesh densities
lc1 = 2e-2;      # Enclosure & core outer
lc2 = 1e-2;      # Core inner
lc3 = 1e-2;      # HV windings
lc4 = 1e-2;      # LV windings

In [ ]:
function gmsh_add_rectangle(mid, width, height, lc)
    geo = gmsh.model.geo;
    
    # Corner points
    p1 = geo.addPoint(mid[1] - width / 2, mid[2] - height / 2, 0, lc);
    p2 = geo.addPoint(mid[1] + width / 2, mid[2] - height / 2, 0, lc);
    p3 = geo.addPoint(mid[1] + width / 2, mid[2] + height / 2, 0, lc);
    p4 = geo.addPoint(mid[1] - width / 2, mid[2] + height / 2, 0, lc);
    points = [p1, p2, p3, p4];
    
    # Lines
    l1 = geo.addLine(p1, p2);
    l2 = geo.addLine(p2, p3);
    l3 = geo.addLine(p3, p4);
    l4 = geo.addLine(p4, p1);
    lines = [l1, l2, l3, l4];
    
    # Curve loop
    loop = geo.addCurveLoop(lines);
    
    return loop, lines, points;
end

In [ ]:
function gmsh_add_rectangle(mid, width, height, lc, radius)
    geo = gmsh.model.geo;
    
    # Corner points
    p1 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] - height / 2, 0, lc);
    p2 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] - height / 2, 0, lc);
    p3 = geo.addPoint(mid[1] + width / 2         , mid[2] - height / 2 + radius, 0, lc);
    p4 = geo.addPoint(mid[1] + width / 2         , mid[2] + height / 2 - radius, 0, lc);
    p5 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] + height / 2, 0, lc);
    p6 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] + height / 2, 0, lc);
    p7 = geo.addPoint(mid[1] - width / 2         , mid[2] + height / 2 - radius, 0, lc);
    p8 = geo.addPoint(mid[1] - width / 2         , mid[2] - height / 2 + radius, 0, lc);
    
    c1 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] - height / 2 + radius, 0, 1);
    c2 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] - height / 2 + radius, 0, 1);
    c3 = geo.addPoint(mid[1] + width / 2 - radius, mid[2] + height / 2 - radius, 0, 1);
    c4 = geo.addPoint(mid[1] - width / 2 + radius, mid[2] + height / 2 - radius, 0, 1);
    points = [p1, p2, p3, p4, p5, p6, p7, p8];
    
    # Lines
    l1 = geo.addLine(p1, p2)
    l2 = geo.addCircleArc(p2, c2, p3)
    l3 = geo.addLine(p3, p4)
    l4 = geo.addCircleArc(p4, c3, p5)
    l5 = geo.addLine(p5, p6)
    l6 = geo.addCircleArc(p6, c4, p7)
    l7 = geo.addLine(p7, p8)
    l8 = geo.addCircleArc(p8, c1, p1)
    lines = [l1, l2, l3, l4, l5, l6, l7, l8]
    
    # Curve loop
    loop = geo.addCurveLoop(lines);
    
    return loop, lines, points;
end

In [ ]:
gmsh.model.add("transformer")
geo = gmsh.model.geo;

## Enclosure
enclosure_lp, enclosure_lines, _ = gmsh_add_rectangle([0, 0.5 * (hencl1 + hencl2)], wencl, hencl1 - hencl2, lc1)

## Core 
core_lp, _, _  = gmsh_add_rectangle([0, 0], wcore, hcore, lc1)     # Core outline
cgap1_lp, _, _ = gmsh_add_rectangle([-mgap, 0], wgap, hgap, lc2, 1.5e-2)   # Gap left
cgap2_lp, _, _ = gmsh_add_rectangle([+mgap, 0], wgap, hgap, lc2, 1.5e-2)   # Gap right

## HV windings
xm = mgap + wgap / 2 + (wcore / 2 - mgap - wgap / 2) / 2;

hv1l_lp, _, _ = gmsh_add_rectangle([-xm - mwhv, 0], wwhv, hwhv, lc3)
hv1r_lp, _, _ = gmsh_add_rectangle([-xm + mwhv, 0], wwhv, hwhv, lc3)
hv2l_lp, _, _ = gmsh_add_rectangle([    - mwhv, 0], wwhv, hwhv, lc3)
hv2r_lp, _, _ = gmsh_add_rectangle([    + mwhv, 0], wwhv, hwhv, lc3)
hv3l_lp, _, _ = gmsh_add_rectangle([+xm - mwhv, 0], wwhv, hwhv, lc3)
hv3r_lp, _, _ = gmsh_add_rectangle([+xm + mwhv, 0], wwhv, hwhv, lc3)

## LV windings
xm = mgap + wgap / 2 + (wcore / 2 - mgap - wgap / 2) / 2;

lv1l_lp, _, _ = gmsh_add_rectangle([-xm - mwlv, 0], wwlv, hwlv, lc4)
lv1r_lp, _, _ = gmsh_add_rectangle([-xm + mwlv, 0], wwlv, hwlv, lc4)
lv2l_lp, _, _ = gmsh_add_rectangle([    - mwlv, 0], wwlv, hwlv, lc4)
lv2r_lp, _, _ = gmsh_add_rectangle([    + mwlv, 0], wwlv, hwlv, lc4)
lv3l_lp, _, _ = gmsh_add_rectangle([+xm - mwlv, 0], wwlv, hwlv, lc4)
lv3r_lp, _, _ = gmsh_add_rectangle([+xm + mwlv, 0], wwlv, hwlv, lc4)

## Surfaces
geo.addPlaneSurface([enclosure_lp, core_lp, hv1l_lp, lv1l_lp, hv3r_lp, lv3r_lp], 1)
geo.addPlaneSurface([core_lp, cgap1_lp, cgap2_lp], 2)
geo.addPlaneSurface([cgap1_lp, hv1r_lp, lv1r_lp, hv2l_lp, lv2l_lp], 3)
geo.addPlaneSurface([cgap2_lp, hv2r_lp, lv2r_lp, hv3l_lp, lv3l_lp], 4)

geo.addPlaneSurface([hv1l_lp], 5)
geo.addPlaneSurface([hv1r_lp], 6)
geo.addPlaneSurface([hv2l_lp], 7)
geo.addPlaneSurface([hv2r_lp], 8)
geo.addPlaneSurface([hv3l_lp], 9)
geo.addPlaneSurface([hv3r_lp], 10)

geo.addPlaneSurface([lv1l_lp], 11)
geo.addPlaneSurface([lv1r_lp], 12)
geo.addPlaneSurface([lv2l_lp], 13)
geo.addPlaneSurface([lv2r_lp], 14)
geo.addPlaneSurface([lv3l_lp], 15)
geo.addPlaneSurface([lv3r_lp], 16)

geo.synchronize()

## Physical Groups
geo.addPhysicalGroup(2, [1, 3, 4], 1)   # Transformer oil
geo.addPhysicalGroup(2, [2], 2)         # Core
geo.addPhysicalGroup(2, [5], 3)         # HV winding phase 1 left
geo.addPhysicalGroup(2, [6], 4)         # HV winding phase 1 right
geo.addPhysicalGroup(2, [7], 5)         # HV winding phase 2 left
geo.addPhysicalGroup(2, [8], 6)         # HV winding phase 2 right
geo.addPhysicalGroup(2, [9], 7)         # HV winding phase 3 left
geo.addPhysicalGroup(2, [10], 8)        # HV winding phase 3 right

geo.addPhysicalGroup(2, [11], 9)        # LV winding phase 1 left
geo.addPhysicalGroup(2, [12], 10)       # LV winding phase 1 right
geo.addPhysicalGroup(2, [13], 11)       # LV winding phase 2 left
geo.addPhysicalGroup(2, [14], 12)       # LV winding phase 2 right
geo.addPhysicalGroup(2, [15], 13)       # LV winding phase 3 left
geo.addPhysicalGroup(2, [16], 14)       # LV winding phase 3 right

geo.addPhysicalGroup(2, [5, 6, 7, 8, 9, 10], 15)      # HV windings
geo.addPhysicalGroup(2, [11, 12, 13, 14, 15, 16], 16) # LV windings

geo.addPhysicalGroup(1, enclosure_lines, 1)  # Enclosure boundary

gmsh.model.setPhysicalName(2, 1, "Oil")
gmsh.model.setPhysicalName(2, 2, "Core")
gmsh.model.setPhysicalName(2, 15, "HV windings")
gmsh.model.setPhysicalName(2, 16, "LV windings")

gmsh.model.setPhysicalName(1, 1, "Enclosure")

# Generate Mesh
geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("geo/transformer_stedin.msh")

![Geometry](images/transformer/geometry.png)

# Load Geometry
To provide efficient access to the geometry data, we pre-process it and store it in a `mesh_data` struct. This contains information about the nodes (number and position), as well as the elements (number, connectivity, and physical domain).

In [4]:
# mesh_data struct
#  saves the nodes and elements of a gmsh geometry, for easy passing around in functions
struct mesh_data
    nnodes  # number of nodes
    xnode   # array of x coordinates
    ynode   # array of y coordinates
    
    nelements   # number of elements
    #element_connectivity  # array of connectivity for each element
    e_group     # array containing the physical group number of each element
    elements    # <new addition> more conveniently structured connectivity array
end

# Loads nodes, elements, and element physical groups from gmsh and stores them in a mesh_data struct
function get_mesh_data()
    #..2/11 Get and sort the mesh nodes
    #..Observe that although the mesh is two-dimensional,
    #..the z-coordinate that is equal to zero is stored as well.
    #..Observe that the coordinates are stored contiguously for computational
    #..efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    
    #..4/12 Get the mesh elements
    #..observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelements = length(element_ids[1])

    #..5/12 Create groups of elements for the subdomains
    #..for loop that creates a vector describing which physical group an element belongs to
    ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 1)
    ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 2)
    ngroup3 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 3)
    ngroup4 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 4)
    ngroup5 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 5)
    ngroup6 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 6)
    ngroup7 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 7)
    ngroup8 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 8)
    ngroup9 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 9)
    ngroup10 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 10)
    ngroup11 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 11)
    ngroup12 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 12)
    ngroup13 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 13)
    ngroup14 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 14)
    ngroup15 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 15)
    ngroup16 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 16)
    e_group = zeros(1,nelements)
    
    elements = [zeros(Int, 3) for i in 1:nelements];
    
    for element_id in 1:nelements
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        
        # Store connectivity in a convenient format
        elements[element_id] = [node1_id, node2_id, node3_id];
        
        # Determine which physical group the element belongs to
        G1  = sum(node1_id.== ngroup1[1])+sum(node2_id.== ngroup1[1])+sum(node3_id.== ngroup1[1]) # Oil
        G2  = sum(node1_id.== ngroup2[1])+sum(node2_id.== ngroup2[1])+sum(node3_id.== ngroup2[1]) # Core
        G3  = sum(node1_id.== ngroup3[1])+sum(node2_id.== ngroup3[1])+sum(node3_id.== ngroup3[1]) # HV winding phase 1 left
        G4  = sum(node1_id.== ngroup4[1])+sum(node2_id.== ngroup4[1])+sum(node3_id.== ngroup4[1]) # HV winding phase 1 right
        G5  = sum(node1_id.== ngroup5[1])+sum(node2_id.== ngroup5[1])+sum(node3_id.== ngroup5[1]) # HV winding phase 2 left
        G6  = sum(node1_id.== ngroup6[1])+sum(node2_id.== ngroup6[1])+sum(node3_id.== ngroup6[1]) # HV winding phase 2 right
        G7  = sum(node1_id.== ngroup7[1])+sum(node2_id.== ngroup7[1])+sum(node3_id.== ngroup7[1]) # HV winding phase 3 left
        G8  = sum(node1_id.== ngroup8[1])+sum(node2_id.== ngroup8[1])+sum(node3_id.== ngroup8[1]) # HV winding phase 3 right
        G9  = sum(node1_id.== ngroup9[1])+sum(node2_id.== ngroup9[1])+sum(node3_id.== ngroup9[1]) # LV winding phase 1 left
        G10 = sum(node1_id.== ngroup10[1])+sum(node2_id.== ngroup10[1])+sum(node3_id.== ngroup10[1]) # LV winding phase 1 right
        G11 = sum(node1_id.== ngroup11[1])+sum(node2_id.== ngroup11[1])+sum(node3_id.== ngroup11[1]) # LV winding phase 2 left
        G12 = sum(node1_id.== ngroup12[1])+sum(node2_id.== ngroup12[1])+sum(node3_id.== ngroup12[1]) # LV winding phase 2 right
        G13 = sum(node1_id.== ngroup13[1])+sum(node2_id.== ngroup13[1])+sum(node3_id.== ngroup13[1]) # LV winding phase 3 left
        G14 = sum(node1_id.== ngroup14[1])+sum(node2_id.== ngroup14[1])+sum(node3_id.== ngroup14[1]) # LV winding phase 3 right
        
        if G1 == 3
            e_group[element_id] = 1;
        elseif G2 == 3
            e_group[element_id] = 2;
        elseif G3 == 3
            e_group[element_id] = 3;
        elseif G4 == 3
            e_group[element_id] = 4;
        elseif G5 == 3
            e_group[element_id] = 5;
        elseif G6 == 3
            e_group[element_id] = 6;
        elseif G7 == 3
            e_group[element_id] = 7;
        elseif G8 == 3
            e_group[element_id] = 8;
        elseif G9 == 3
            e_group[element_id] = 9;
        elseif G10 == 3
            e_group[element_id] = 10;
        elseif G11 == 3
            e_group[element_id] = 11;
        elseif G12 == 3
            e_group[element_id] = 12;
        elseif G13 == 3
            e_group[element_id] = 13;
        elseif G14 == 3
            e_group[element_id] = 14;
        end
    end
    
    return mesh_data(nnodes, xnode, ynode, nelements, e_group, elements)
end

get_mesh_data (generic function with 1 method)

# Core Losses
The core losses, also called _iron losses_, are the losses occurring in the magnetic core material due to hysteresis and eddy currents.

## Steinmetz Equation
The core losses can be modeled in a limited range of operating frequency, flux density, and temperature using the Steinmetz equation. This equation provides an _empirical_ model of the variation of core losses with excitation frequency $f$ and flux density swing $\Delta B$. The coefficients $k$, $\alpha$, and $\beta$ are typically obtained through curve fitting the datasheet graphs of core loss versus frequency, temperature, and flux density swing.
$$ P_v = k f^\alpha (\Delta B)^\beta \qquad \mathrm{[W/m^3]} $$
The equation provides a power loss per unit volume, which can be translated to total loss by integrating this value over the volume of the magnetic core. In doing so, the lamination factor of the core must be taken into account. This reduces the effective volume.

Typical values are $\alpha \approx 1 - 2$ and $\beta \approx 2 - 3$. The value of $k$ depends heavily on the material and the chosen values of $\alpha$ and $\beta$.

## Steinmetz Parameters
The manufacturer of the transformer core material does not specify core loss graphs in the [datasheet (30JG130)](https://www.jfe-steel.co.jp/en/products/electrical/catalog/f1e-001.pdf). However, they do give typical core loss values at $f = 50\,\mathrm{Hz}$ and $f = 60\,\mathrm{Hz}$. This allows us to make an estimate of the Steinmetz parameters.
$$ \left\{ \begin{array}{l}
    P_1 = (k B^\beta) \cdot f_1^\alpha \\
    P_2 = (k B^\beta) \cdot f_2^\alpha
\end{array} \right. \implies \alpha = \frac{\log\left( \frac{P_1}{P_2} \right)}{\log\left( \frac{f_1}{f_2} \right)} \approx 1.53 $$
The exponent $\beta$ cannot be derived from the available data, so it is estimated to be $\approx 1.6$. From this, we can finally calculate the parameter $k$ by taking the given core loss at $B = 1.7\,\mathrm{T}$.
$$ k = \frac{P}{f^\alpha B^\beta} \approx 2.25\,\mathrm{W/(Hz^\alpha \cdot T^\beta \cdot m^3)} $$

## Open-circuit Test
An open-circuit test is performed by the transformer manufacturer to determine the core losses. During this test, the primary windings are open-circuited and the nominal secondary voltage is applied to the secondary windings. Because the primary windings do not provide an opposing flux in this configuration, the transformer core is driven into saturation very easily with a limited amount of secondary current. Therefore, the measured losses are nearly equal to the core losses.

In [5]:
# Pre-calculate the matrix contributions that are constant in the non-linear problem
function precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega, nc)
    elids = 1:mshdata.nelements;
    
    Bmat = [zeros(9) for i in elids];
    Cmat = [zeros(Complex{Float64}, 9) for i in elids];
    Emat = [zeros(3,3) for i in elids];
    
    
    CFF  = 0.3;
    Vsec = 420 * sqrt(2 / 3);
    Rsec = 1.2999e-3 / CFF;
    Asec = Awlv;
    lsec = 0.4;
    
    ceqs_r = zeros(Complex{Float64}, nc, mshdata.nnodes);
    ceqs_c = zeros(Complex{Float64}, mshdata.nnodes, nc);
    
    f = zeros(Complex{Float64}, mshdata.nnodes + nc)
    
    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    #..8/12 Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        #....retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Eloc = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Eloc[3,:] .= 0;
        Bloc = area_id*(transpose(Eloc)*Eloc);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        f[nodes] += floc;
        
        # LV winding 1 left (positive)
        if(mshdata.e_group[element_id] == 9)
            ceqs_c[nodes, 1] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[1, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            ceqs_c[nodes, 1] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[1, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 11)
            ceqs_c[nodes, 2] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[2, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            ceqs_c[nodes, 2] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[2, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 13)
            ceqs_c[nodes, 3] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[3, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            ceqs_c[nodes, 3] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[3, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
        end
        
        Bmat[element_id] = Bloc[:];
        Cmat[element_id] = Cloc[:];
        Emat[element_id] = Eloc;
    end
    
    f[mshdata.nnodes + 1] = Vsec * exp(1im * 2pi/3) / Rsec;
    f[mshdata.nnodes + 2] = Vsec / Rsec;
    f[mshdata.nnodes + 3] = Vsec * exp(1im * -2pi/3) / Rsec;
    
    return Bmat, Cmat, Emat, f, ceqs_r, ceqs_c;
end

precalc_contribs (generic function with 1 method)

In [17]:
function process_voltage(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    Nnodes = mshdata.nnodes;
    
    Bx = zeros(Complex{Float64}, mshdata.nelements);
    By = zeros(Complex{Float64}, mshdata.nelements);
    
    Jel = zeros(Complex{Float64}, mshdata.nelements);
    
    kv = 2.25; a = 1.53; b = 1.6;
    kl = 0.955; # Lamination fill factor
    Pv = zeros(mshdata.nelements);
    Pcore = 0;
    K  = kv * 50^a;
    L  = 0.4;
    
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Get nodes constituting the element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
        
        # Get x and y coordinates of the three nodes
        xnode = mshdata.xnode; ynode = mshdata.ynode;
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Calculate shape function parameters
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    
        # Calculate Bx and By from the solution coefficients and the shape function parameters
        c = u[[node1_id, node2_id, node3_id]];
        Bx[element_id] = sum(c .* Emat[2,:]);
        By[element_id] = -sum(c .* Emat[1,:]);
        
        # Calculate eddy current loss
        sigma = conductivityperelement[element_id];
        Jel[element_id] = sourceperelement[element_id] + 1im * omega * sigma * 1/3 * sum(c);
        
        # Calculate core loss
        Pv[element_id] = K * sqrt(norm(Bx[element_id])^2 + norm(By[element_id])^2)^b;
        Pcore = Pcore + Pv[element_id] * area_id * L * kl;
        
        # LV winding 1 left (positive)
        if(mshdata.e_group[element_id] == 9)
            Jel[element_id] += u[Nnodes + 1] * Ns / Awlv;
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            Jel[element_id] += -u[Nnodes + 1] * Ns / Awlv;
            
        # LV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 11)
            Jel[element_id] += u[Nnodes + 2] * Ns / Awlv;
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            Jel[element_id] += -u[Nnodes + 2] * Ns / Awlv;
            
        # LV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 13)
            Jel[element_id] += u[Nnodes + 3] * Ns / Awlv;
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            Jel[element_id] += -u[Nnodes + 3] * Ns / Awlv;
        end
    end
    
    Jel = norm.(Jel);
    
    # H is related to B through the reluctivity
    Hx = reluctivityperelement' .* Bx;
    Hy = reluctivityperelement' .* By;
    
    # Energy is 0.5 * dot(B, H)
    Wm = 0.5 * (Bx .* Hx .+ By .* Hy);
    
    return (Bx,By), (Hx, Hy), Wm, Jel, Pv, Pcore;
end

process_voltage (generic function with 1 method)

In [7]:
function assemble_matrices(u, I, J, Bmat, Cmat, Emat, N, nc)
    c = map(el -> u[el], mshdata.elements)
    Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
    By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
    Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
    reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);
    
    # Generate matrix contributions
    Vb = reduce(vcat, map((B, nu) -> B * nu, Bmat, reluctivityperelement));
    Vc = reduce(vcat, Cmat);
    V  = Vb + Vc;
    
    return sparse(I, J, V, N + nc, N + nc);
end

function apply_boundaries(A, bnd_node_ids, e)
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    #f[bnd_node_ids] .= 0;
    
    return A#, f;
end

function apply_circuits(A, N, nc, ceqs_r, ceqs_c)
    idx_i = N .+ (1:nc);
    idx_j = 1:N;
    
    A[idx_i, idx_j] = ceqs_r;
    A[idx_j, idx_i] = ceqs_c;
    A[idx_i, idx_i] = Diagonal(ones(nc));
    
    return A;
end

apply_circuits (generic function with 1 method)

In [8]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

Info    : Reading 'geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading 'geo/transformer_stedin.msh'


In [9]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
# Because the current is now determined by the circuit coupling, we set the imposed current density to zero.
sourcefunction(group_id) = 0
sourceperelement = map(sourcefunction, mshdata.e_group);

# Permeability function
bh_a = 1 / 47e3;
bh_b = 3.6;
bh_c = 2.1e8;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

fmu(group_id, B) = mu0 + (fmur_core(B) - 1) * mu0 * (group_id == 2);
reluctivityfunction(group_id, B) = 1 / fmu(group_id, B);

sigma_core = 0;
conductivityfunction(group_id) = sigma_core * (group_id == 2);
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [10]:
Bmat, Cmat, Emat, f, ceqs_r, ceqs_c = precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega, 3);

# Generate index vectors
I_ = reduce(vcat, view.(mshdata.elements, Ref([1, 2, 3, 1, 2, 3, 1, 2, 3])))
J_ = reduce(vcat, view.(mshdata.elements, Ref([1, 1, 1, 2, 2, 2, 3, 3, 3])))
N = mshdata.nnodes;

# Generate vectors for boundary condition 
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
e = Diagonal(ones(size(bnd_node_ids)));

In [11]:
function nonlinsolve(mshdata, I, J, Bmat, Cmat, Emat, ceqs_r, ceqs_c, N, bnd_node_ids, e, alpha)
    A = assemble_matrices(zeros(mshdata.nnodes), I, J, Bmat, Cmat, Emat, N, 3);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    f[bnd_node_ids] .= 0;
    A = apply_circuits(A, N, 3, ceqs_r, ceqs_c);

    u = A \ f;

    du = 1e6;
    Niter = 1;

    tol = 1e-6;
    Nmax = 200;
    
    uhist = zeros(N + 3);
    
    while (du > tol) && (Niter < Nmax)
        uprev = u;
        uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

        A = assemble_matrices(uhist, I, J, Bmat, Cmat, Emat, N, 3);
        A = apply_boundaries(A, bnd_node_ids, e);
        A = apply_circuits(A, N, 3, ceqs_r, ceqs_c);
        
        u = A \ f;

        du = norm(u - uprev);
        Niter += 1;
        print("#$Niter: $du\n")
    end
    
    return u;
end

nonlinsolve (generic function with 1 method)

In [18]:
u = nonlinsolve(mshdata, I_, J_, Bmat, Cmat, Emat, ceqs_r, ceqs_c, N, bnd_node_ids, e, 0.9);

#2: 1.0024379939677688e-5
#3: 0.0010078759922119774
#4: 0.011945683866705331
#5: 0.054520722737901345
#6: 0.13221248509651135
#7: 0.20985747158481488
#8: 0.2784221387891537
#9: 0.3455899788438856
#10: 0.4126902006429294
#11: 0.4814169195017862
#12: 0.5531739637434913
#13: 0.6275521318035236
#14: 0.7018664952521333
#15: 0.7718418274605218
#16: 0.8330499754956072
#17: 0.8822028016519808
#18: 0.9176355046956034
#19: 0.9390790265824757
#20: 0.9472048644355991
#21: 0.9432483396851248
#22: 0.9287562685258169
#23: 0.9054146248119243
#24: 0.8749236211872637
#25: 0.8389083670685465
#26: 0.7988599865248392
#27: 0.7561020108249925
#28: 0.7117761548581731
#29: 0.6668420004444835
#30: 0.6220858175314297
#31: 0.5781350246972409
#32: 0.5354756007497098
#33: 0.49447053014854714
#34: 0.455378093345801
#35: 0.41836904132297115
#36: 0.38354236785826135
#37: 0.35093927210945725
#38: 0.32055532214542326
#39: 0.29235091115800477
#40: 0.26626001691940937
#41: 0.24219747822506268
#42: 0.22006498554494056
#43:

In [19]:
c = map(el -> u[el], mshdata.elements)
Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);

B, H, Wm, Jel, Pv, Pcore = process_voltage(mshdata, u, sourceperelement, reluctivityperelement', conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));
Hnorm = norm.(sqrt.(H[1].^2 + H[2].^2));

Isec1 = u[mshdata.nnodes + 1];
Isec2 = u[mshdata.nnodes + 2];
Isec3 = u[mshdata.nnodes + 3];
Az = u[1:mshdata.nnodes];

In [20]:
print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

I_{sec1}: 27.233244263952965 A, 16.388677996995593 deg
I_{sec2}: 15.354574376576858 A, -89.89894601705129 deg
I_{sec3}: 27.16562606354019 A, 163.63412193161528 deg


In [21]:
print("Core  loss: $Pcore W\n")

Core  loss: 1099.9481303350703 W


In [22]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer_loss/sim_core.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(Az);
vtkfile["imA", VTKPointData()]  = imag.(Az);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Hnorm", VTKCellData()] = Hnorm;
vtkfile["Jel", VTKCellData()]   = Jel;
vtkfile["mu_r", VTKCellData()]  = 1 ./ (mu0 * reluctivityperelement);
vtkfile["Pv", VTKCellData()]    = Pv;

# Save the file
outfiles = vtk_save(vtkfile);

![Open-Circuit Test: Flux Density](images/transformer_loss/oc_normB.png)

![Open-Circuit Test: Volumetric Loss](images/transformer_loss/oc_Pv.png)

# Winding Losses
Short-circuit test: secondary windings shorted, low voltage applied to the primary windings.

In [40]:
# Pre-calculate the matrix contributions that are constant in the non-linear problem
function precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega, nc)
    elids = 1:mshdata.nelements;
    
    Bmat = [zeros(9) for i in elids];
    Cmat = [zeros(Complex{Float64}, 9) for i in elids];
    Emat = [zeros(3,3) for i in elids];
    
    CFFp = 0.3;
    Vpri = 460.3 * sqrt(2);
    Rpri = 1.8131 / CFFp;
    Apri = Awhv;
    lpri = 0.4;
    
    CFFs = 0.3;
    Vsec = 0;
    Rsec = 1.2999e-3 / CFFs;
    Asec = Awlv;
    lsec = 0.4;
    
    ceqs_r = zeros(Complex{Float64}, nc, mshdata.nnodes);
    ceqs_c = zeros(Complex{Float64}, mshdata.nnodes, nc);
    
    f = zeros(Complex{Float64}, mshdata.nnodes + nc)
    
    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    #..8/12 Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        #....retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Eloc = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Eloc[3,:] .= 0;
        Bloc = area_id*(transpose(Eloc)*Eloc);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        f[nodes] += floc;
        
        # HV winding 1 left (negative)
        if(mshdata.e_group[element_id] == 3)
            ceqs_c[nodes, 4] += -Np / Apri * area_id/3 * [1;1;1];
            ceqs_r[4, nodes] += 1im * omega * Np * lpri / (Apri * Rpri) * area_id/3 * [1;1;1];
            
        # HV winding 1 right (positive)
        elseif(mshdata.e_group[element_id] == 4)
            ceqs_c[nodes, 4] += Np / Apri * area_id/3 * [1;1;1];
            ceqs_r[4, nodes] += -1im * omega * Np * lpri / (Apri * Rpri) * area_id/3 * [1;1;1];
            
        # HV winding 2 left (negative)
        elseif(mshdata.e_group[element_id] == 5)
            ceqs_c[nodes, 5] += -Np / Apri * area_id/3 * [1;1;1];
            ceqs_r[5, nodes] += 1im * omega * Np * lpri / (Apri * Rpri) * area_id/3 * [1;1;1];
            
        # HV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 6)
            ceqs_c[nodes, 5] += Np / Apri * area_id/3 * [1;1;1];
            ceqs_r[5, nodes] += -1im * omega * Np * lpri / (Apri * Rpri) * area_id/3 * [1;1;1];
            
        # HV winding 3 left (negative)
        elseif(mshdata.e_group[element_id] == 7)
            ceqs_c[nodes, 6] += -Np / Apri * area_id/3 * [1;1;1];
            ceqs_r[6, nodes] += 1im * omega * Np * lpri / (Apri * Rpri) * area_id/3 * [1;1;1];
            
        # HV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 8)
            ceqs_c[nodes, 6] += Np / Apri * area_id/3 * [1;1;1];
            ceqs_r[6, nodes] += -1im * omega * Np * lpri / (Apri * Rpri) * area_id/3 * [1;1;1];
            
        # LV winding 1 left (positive)
        elseif(mshdata.e_group[element_id] == 9)
            ceqs_c[nodes, 1] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[1, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            ceqs_c[nodes, 1] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[1, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 left (positive)
        elseif(mshdata.e_group[element_id] == 11)
            ceqs_c[nodes, 2] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[2, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            ceqs_c[nodes, 2] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[2, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 left (positive)
        elseif(mshdata.e_group[element_id] == 13)
            ceqs_c[nodes, 3] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[3, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            ceqs_c[nodes, 3] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[3, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
        end
        
        Bmat[element_id] = Bloc[:];
        Cmat[element_id] = Cloc[:];
        Emat[element_id] = Eloc;
    end
    
    f[mshdata.nnodes + 1] = Vsec * exp(1im * 2pi/3) / Rsec;
    f[mshdata.nnodes + 2] = Vsec / Rsec;
    f[mshdata.nnodes + 3] = Vsec * exp(1im * -2pi/3) / Rsec;
    
    f[mshdata.nnodes + 4] = Vpri * exp(1im * 2pi/3) / Rpri;
    f[mshdata.nnodes + 5] = Vpri / Rpri;
    f[mshdata.nnodes + 6] = Vpri * exp(1im * -2pi/3) / Rpri;
    
    return Bmat, Cmat, Emat, f, ceqs_r, ceqs_c;
end

precalc_contribs (generic function with 1 method)

In [55]:
function process_voltage(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    Nnodes = mshdata.nnodes;
    
    Bx = zeros(Complex{Float64}, mshdata.nelements);
    By = zeros(Complex{Float64}, mshdata.nelements);
    
    Jel = zeros(Complex{Float64}, mshdata.nelements);
    
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Get nodes constituting the element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
        
        # Get x and y coordinates of the three nodes
        xnode = mshdata.xnode; ynode = mshdata.ynode;
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Calculate shape function parameters
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    
        # Calculate Bx and By from the solution coefficients and the shape function parameters
        c = u[[node1_id, node2_id, node3_id]];
        Bx[element_id] = sum(c .* Emat[2,:]);
        By[element_id] = -sum(c .* Emat[1,:]);
        
        # Calculate eddy current loss
        sigma = conductivityperelement[element_id];
        Jel[element_id] = sourceperelement[element_id] + 1im * omega * sigma * 1/3 * sum(c);
        
        # HV winding 1 left (negative)
        if(mshdata.e_group[element_id] == 3)
            Jel[element_id] += -u[Nnodes + 4] * Np / Awhv;
            
        # HV winding 1 right (positive)
        elseif(mshdata.e_group[element_id] == 4)
            Jel[element_id] += u[Nnodes + 4] * Np / Awhv;
            
        # HV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 5)
            Jel[element_id] += -u[Nnodes + 5] * Np / Awhv;
            
        # HV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 6)
            Jel[element_id] += u[Nnodes + 5] * Np / Awhv;
            
        # HV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 7)
            Jel[element_id] += -u[Nnodes + 6] * Np / Awhv;
            
        # HV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 8)
            Jel[element_id] += u[Nnodes + 6] * Np / Awhv;
              
        # LV winding 1 left (positive)
        elseif(mshdata.e_group[element_id] == 9)
            Jel[element_id] += u[Nnodes + 1] * Ns / Awlv;
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            Jel[element_id] += -u[Nnodes + 1] * Ns / Awlv;
            
        # LV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 11)
            Jel[element_id] += u[Nnodes + 2] * Ns / Awlv;
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            Jel[element_id] += -u[Nnodes + 2] * Ns / Awlv;
            
        # LV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 13)
            Jel[element_id] += u[Nnodes + 3] * Ns / Awlv;
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            Jel[element_id] += -u[Nnodes + 3] * Ns / Awlv;
        end
    end
    
    Jel = norm.(Jel);
    
    # H is related to B through the reluctivity
    Hx = reluctivityperelement' .* Bx;
    Hy = reluctivityperelement' .* By;
    
    # Energy is 0.5 * dot(B, H)
    Wm = 0.5 * (Bx .* Hx .+ By .* Hy);
    
    return (Bx,By), (Hx, Hy), Wm, Jel;
end

process_voltage (generic function with 1 method)

In [41]:
function assemble_matrices(u, I, J, Bmat, Cmat, Emat, N, nc)
    c = map(el -> u[el], mshdata.elements)
    Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
    By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
    Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
    reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);
    
    # Generate matrix contributions
    Vb = reduce(vcat, map((B, nu) -> B * nu, Bmat, reluctivityperelement));
    Vc = reduce(vcat, Cmat);
    V  = Vb + Vc;
    
    return sparse(I, J, V, N + nc, N + nc);
end

function apply_boundaries(A, bnd_node_ids, e)
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    #f[bnd_node_ids] .= 0;
    
    return A#, f;
end

function apply_circuits(A, N, nc, ceqs_r, ceqs_c)
    idx_i = N .+ (1:nc);
    idx_j = 1:N;
    
    A[idx_i, idx_j] = ceqs_r;
    A[idx_j, idx_i] = ceqs_c;
    A[idx_i, idx_i] = Diagonal(ones(nc));
    
    return A;
end

apply_circuits (generic function with 1 method)

In [42]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

Info    : Reading 'geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading 'geo/transformer_stedin.msh'


In [43]:
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# Source current density J
# Because the current is now determined by the circuit coupling, we set the imposed current density to zero.
sourcefunction(group_id) = 0
sourceperelement = map(sourcefunction, mshdata.e_group);

# Permeability function
bh_a = 1 / 47e3;
bh_b = 3.6;
bh_c = 2.1e8;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

fmu(group_id, B) = mu0 + (fmur_core(B) - 1) * mu0 * (group_id == 2);
reluctivityfunction(group_id, B) = 1 / fmu(group_id, B);

sigma_core = 0;
conductivityfunction(group_id) = sigma_core * (group_id == 2);
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [44]:
Bmat, Cmat, Emat, f, ceqs_r, ceqs_c = precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega, 6);

# Generate index vectors
I_ = reduce(vcat, view.(mshdata.elements, Ref([1, 2, 3, 1, 2, 3, 1, 2, 3])))
J_ = reduce(vcat, view.(mshdata.elements, Ref([1, 1, 1, 2, 2, 2, 3, 3, 3])))
N = mshdata.nnodes;

# Generate vectors for boundary condition 
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
e = Diagonal(ones(size(bnd_node_ids)));

In [45]:
function nonlinsolve(mshdata, I, J, Bmat, Cmat, Emat, ceqs_r, ceqs_c, N, bnd_node_ids, e, alpha)
    A = assemble_matrices(zeros(mshdata.nnodes), I, J, Bmat, Cmat, Emat, N, 6);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    f[bnd_node_ids] .= 0;
    A = apply_circuits(A, N, 6, ceqs_r, ceqs_c);

    u = A \ f;

    du = 1e6;
    Niter = 1;

    tol = 1e-6;
    Nmax = 200;
    
    uhist = zeros(N + 6);
    
    while (du > tol) && (Niter < Nmax)
        uprev = u;
        uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

        A = assemble_matrices(uhist, I, J, Bmat, Cmat, Emat, N, 6);
        A = apply_boundaries(A, bnd_node_ids, e);
        A = apply_circuits(A, N, 6, ceqs_r, ceqs_c);
        
        u = A \ f;

        du = norm(u - uprev);
        Niter += 1;
        print("#$Niter: $du\n")
    end
    
    return u;
end

nonlinsolve (generic function with 1 method)

In [58]:
u = nonlinsolve(mshdata, I_, J_, Bmat, Cmat, Emat, ceqs_r, ceqs_c, N, bnd_node_ids, e, 0.9);

#2: 5.948966412542329e-17


In [59]:
c = map(el -> u[el], mshdata.elements)
Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);

B, H, Wm, Jel = process_voltage(mshdata, u, sourceperelement, reluctivityperelement', conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));
Hnorm = norm.(sqrt.(H[1].^2 + H[2].^2));

Isec1 = u[mshdata.nnodes + 1];
Isec2 = u[mshdata.nnodes + 2];
Isec3 = u[mshdata.nnodes + 3];

Ipri1 = u[mshdata.nnodes + 4];
Ipri2 = u[mshdata.nnodes + 5];
Ipri3 = u[mshdata.nnodes + 6];

u = u[1:mshdata.nnodes];

In [72]:
Rpri = 1.8131;
Rsec = 1.2999e-3;

In [78]:
Ppri = norm(Ipri1)^2 * Rpri/2 + norm(Ipri2)^2 * Rpri/2 + norm(Ipri3)^2 * Rpri/2;
Psec = norm(Isec1)^2 * Rsec/2 + norm(Isec2)^2 * Rsec/2 + norm(Isec3)^2 * Rsec/2;

print("Primary loss: $Ppri W\n")
print("Secondary loss: $Psec W\n")

Primary loss: 5421.37631011725 W
Secondary loss: 7639.340762997476 W


In [60]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer_loss/sim_winding.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Hnorm", VTKCellData()] = Hnorm;
vtkfile["Jel", VTKCellData()]   = Jel;
vtkfile["mu_r", VTKCellData()]  = 1 ./ (mu0 * reluctivityperelement);

# Save the file
outfiles = vtk_save(vtkfile);

![Short-Circuit Test: Flux Density](images/transformer_loss/sc_normB.png)

![Short-Circuit Test: Current Density](images/transformer_loss/sc_normJ.png)